In [11]:
#import libraries and data files
import pandas as pd
import numpy as np
from us_state_abbrev import states
from configs import username
from configs import api_key
#Note will need to do pip install: pip install plotly 

import plotly as py

py.tools.set_credentials_file(username=username, api_key=api_key)

In [12]:
abbv = pd.DataFrame.from_dict(states,orient = 'index', columns= ['state name']).reset_index()
#abbv.columns = [["Code","State Name"]]
abbv.head()

,index,state name
0,AK,Alaska
1,AL,Alabama
2,AR,Arkansas
3,AS,American Samoa
4,AZ,Arizona


In [13]:
#ImportData
df_crime = pd.read_csv("data/gun-violence-data_01-2013_03-2018.csv")
df_crime.head()
df_crime.columns

Index(['incident_id', 'date', 'state', 'city_or_county', 'address', 'n_killed',
       'n_injured', 'incident_url', 'source_url',
       'incident_url_fields_missing', 'congressional_district', 'gun_stolen',
       'gun_type', 'incident_characteristics', 'latitude',
       'location_description', 'longitude', 'n_guns_involved', 'notes',
       'participant_age', 'participant_age_group', 'participant_gender',
       'participant_name', 'participant_relationship', 'participant_status',
       'participant_type', 'sources', 'state_house_district',
       'state_senate_district'],
      dtype='object')

In [14]:
df_crime["year"] = df_crime["date"].str[:4]
df_crimes= df_crime.merge(abbv, how="left",left_on ="state", right_on="state name")


df_crimes.head()



,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,year,index,state name
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN,2013,PA,Pennsylvania
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0,2013,CA,California
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0,2013,OH,Ohio
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0,2013,CO,Colorado
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0,2013,NC,North Carolina


In [19]:
df_crimeg = df_crimes.groupby(["index","year"]).agg({'date' : ['count']})

df_crimeg.columns = df_crimeg.columns.droplevel(1)
df_crimeg.head()

df_crimepiv = df_crimeg.unstack() #fill_value =0.0
df_crimepiv.columns = df_crimepiv.columns.droplevel(0)
df_crimepiv = df_crimepiv.drop(columns=['2013','2018'])
3
        #df_crimepiv = df_crimepiv.apply(pd.to_numeric, errors='ignore')
df_crimepiv["avg"]=df_crimepiv.mean(axis=1)
df_crimepiv["delta"]=((df_crimepiv['2017']-df_crimepiv['2014'])*100)/df_crimepiv['2014']
df_crimepiv["Abs"] = df_crimepiv["delta"].abs()
df_crimepiv["text"] = df_crimepiv["delta"].map('{:,.2f}%'.format)

df_crimepiv = df_crimepiv.sort_values(by=['Abs'],ascending =False)
df_crimepiv.head(30)
#df_crimepiv('Code')

year,2014,2015,2016,2017,avg,delta,Abs,text
index,,,,,,,,
SD,72.0,103.0,144.0,199.0,129.50,176.388889,176.388889,176.39%
AK,146.0,380.0,445.0,317.0,322.00,117.123288,117.123288,117.12%
MA,968.0,1338.0,1544.0,1761.0,1402.75,81.921488,81.921488,81.92%
MN,450.0,410.0,468.0,759.0,521.75,68.666667,68.666667,68.67%
IL,3095.0,3456.0,5055.0,5089.0,4173.75,64.426494,64.426494,64.43%
VT,170.0,117.0,94.0,71.0,113.00,-58.235294,58.235294,-58.24%
KS,381.0,461.0,572.0,578.0,498.00,51.706037,51.706037,51.71%
CO,556.0,788.0,812.0,809.0,741.25,45.503597,45.503597,45.50%
AR,572.0,538.0,720.0,820.0,662.50,43.356643,43.356643,43.36%


In [20]:
#Save Dataframe to file
#df_crimepiv.to_csv("data/Crime_Results.csv")

df_crimepivNonSD = df_crimepiv[df_crimepiv.index != 'SD']

In [21]:
#scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            #[0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]
scl = [[0.0, '#007F00'],[0.25,'#F5F5F5'],[0.75,'#ffb9b9'],[1.0,'#C21807']]

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_crimepiv.index.values,
        z = df_crimepiv['delta'].astype(int),
        locationmode = 'USA-states',
        text = df_crimepiv['text'],
        hoverinfo = 'text+location',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Percentage")
        ) ]

layout = dict(
        title = '2014 - 2017 Change in Gun Crime Rate',
    
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.io.write_image(fig, 'images/Gun Crime Change.png')
py.plotly.iplot( fig, filename='2014 - 2017 Change in Gun Crime Rate' )



In [18]:
#scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            #[0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]
scl = [[0.0, '#007F00'],[0.25,'#F5F5F5'],[0.75,'#ffb9b9'],[1.0,'#C21807']]

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_crimepivNonSD.index.values,
        z = df_crimepivNonSD['delta'].astype(int),
        locationmode = 'USA-states',
        text = df_crimepivNonSD['text'],
        hoverinfo = 'text+location',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Percentage")
        ) ]

layout = dict(
        title = '2014 - 2017 Change in Gun Crime Rate',
    
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.io.write_image(fig, 'images/Gun Crime Change.png')
py.plotly.iplot( fig, filename='2014 - 2017 Change in Gun Crime Rate' )



In [33]:
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]
#scl = [[0.0, '#007F00'],[0.25,'#F5F5F5'],[0.75,'#ffb9b9'],[1.0,'#C21807']]

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_crimepiv.index.values,
        z = df_crimepiv['2017'].astype(int),
        locationmode = 'USA-states',
        #text = df_crimepiv['text'],
        hoverinfo = 'z+location',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Crimes (Thousands)")
        ) ]

layout = dict(
        title = '2017 Gun Crimes by State',
    
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.io.write_image(fig, '2017 Gun Crimes by State.png')
py.plotly.iplot( fig, filename='2017 Gun Crimes by State' )

In [40]:
df_location = df_crimes.groupby(["year","index","city_or_county"], as_index=False).agg({'incident_id' : 'count','latitude' : 'mean','longitude' : 'mean'})
df_location.count()
df_location = df_location.loc[df_location['year'] == '2017']
df_location.head()
#df_location = df_location.loc[df_location['index'] == 'IL']
#df_location.count()
df_location.to_csv('data/crime_location.csv')

In [39]:



#df['text'] = df['airport'] + '' + df['city'] + ', ' + df['state'] + '' + 'Arrivals: ' + df['cnt'].astype(str)

scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

data = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_location['longitude'],
        lat = df_location['latitude'],
        #text = df['text'],
        mode = 'markers',
        marker = dict(
            size = 2,
            opacity = 0.2,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width=.25,
                color='rgba(102, 102, 102)'
            ),
            #colorscale = scl,
            #cmin = 0,
            #color = df['cnt'],
            #cmax = df['cnt'].max(),
            
        ))]

layout = dict(
        title = '2017 Gun Crimes by Location',
        
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig = dict( data=data, layout=layout )
py.io.write_image(fig, '2017 Gun Crimes by Location.png')
py.plotly.iplot( fig, validate=False, filename='2017 Gun Crimes by Location' )